In [1]:
!pip install colab_ssh --upgrade

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -r '/content/drive/Othercomputers/내 컴퓨터/workspace/p4-fr-sorry-math-but-love-you/requirements.txt'
!pip install jamo

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 72 kB 577 kB/s 
     |████████████████████████████████| 10.3 MB 10.7 MB/s 
     |████████████████████████████████| 15.3 MB 39.5 MB/s 
     |████████████████████████████████| 25.4 MB 1.3 MB/s 
     |████████████████████████████████| 9.9 MB 51.2 MB/s 
     |████████████████████████████████| 2.2 MB 22.0 MB/s 
     |████████████████████████████████| 29.2 MB 1.3 MB/s 
     |████████████████████████████████| 27.4 MB 76.9 MB/s 
     |████████████████████████████████| 112 kB 61.6 MB/s 
     |████████████████████████████████| 735.4 MB 15 kB/s 
     |████████████████████████████████| 12.8 MB 1.4 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
     |████████████████████████████████| 2.2 MB 49.8 MB/s 
     |████████████████████████████████| 1.8 MB 43.0 MB/s 
     |████████████████████████████████| 346 kB 49.7 MB/s 
     |████████████████████████████████| 47.6 MB 53.3 MB/s 
   

In [4]:
!unzip '/content/drive/MyDrive/input/ocr_ancient.zip' -d /content/data

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_008.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_009.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_010.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_011.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_012.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_013.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_014.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_015.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_016.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245_문학류_유이양문록41_01_017.png  
  inflating: /content/data/이미지데이터/필사본_245_문학류_유이양문록41_01/필사본_245

In [5]:
# 오류데이터 수정
import os
from glob import glob
json_list = glob(os.path.join('/content/data/이미지데이터','*')) # derectory
for i in json_list:
    tmp_list = glob(os.path.join(i,'*'))
    for j in tmp_list: # image파일

        if j.split('/')[-1][:-8] != i.split('/')[-1]:
            dir_name = i.split('/')[-1]
            img_number = j.split('/')[-1][-8:]
            os.rename(j, '/content/data/이미지데이터/'+dir_name+'/'+dir_name+img_number)

In [6]:
import json, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from glob import glob

from tqdm.auto import tqdm
from jamo import h2j, j2hcj

In [ ]:
!rm -rf /content/data/크랍이미지

## 글자 데이터 구축

In [7]:
from jamo import h2j, j2hcj

excep_word = ['풍', '날','젼','광','황','(']
vocab = set()
exception = set()

w_list = []
h_list = []

dir_list = glob(os.path.join('/content/data/라벨링데이터','*')) # derectory
os.makedirs('/content/data/크랍이미지', exist_ok=True)
chr_idx = 0
for i in tqdm(dir_list): # directory list
    dir_name = i.split('/')[-1]
    dir_path = '/'.join(i.split('/')[:-1])
    json_list = glob(os.path.join(i,'*'))
    for j in json_list: # json 파일 list
        with open(j, 'r', encoding='utf-8-sig') as jfile:
            d = json.load(jfile)
            image_num = d['Image_filename'][-4:]
            image_path = os.path.join(dir_path.replace('라벨링데이터', '이미지데이터'), dir_name, dir_name+image_num+'.png')

            image_arr = np.fromfile(image_path, np.uint8)
            image = cv2.imdecode(image_arr, cv2.IMREAD_COLOR)

            # print(image)
            # image = plt.imread(image_path)
            
            # fig, ax = plt.subplots(figsize=(12,12))
            # ax.imshow(image)
            
            for k in d['Text_Coord']:
                bbox = k['bbox']
                annotate = k['annotate']
                if bbox[2] < 56 or bbox[3] < 56:
                    continue
                pp = image[bbox[1]:bbox[1]+bbox[3],bbox[0]:bbox[0]+bbox[2],:] # 한글자 이미지
                rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3], linewidth=3, edgecolor='r', facecolor='none')
                # ax.add_patch(rect)
                
                try:
                    
                    if len(annotate) == 2:
                        extention = '.png'
                        result, encoded_img = cv2.imencode(extention, pp)
                        if result:
                            if annotate[0] in excep_word or annotate[1] in excep_word:
                                continue
                            with open(f'/content/data/크랍이미지/{dir_name}_{chr_idx:05d}_{annotate[0]}-{annotate[1]}-b_.png', 'wb') as f:
                                encoded_img.tofile(f)
                        vocab.add(annotate[0])
                        vocab.add(annotate[1])
                        vocab.add('b_')
                        chr_idx += 1
                        continue
                    elif len(annotate) == 3:
                        extention = '.png'
                        result, encoded_img = cv2.imencode(extention, pp)
                        if result:
                            if annotate[0] in excep_word or annotate[1] in excep_word or annotate[2] in excep_word:
                                continue
                            with open(f'/content/data/크랍이미지/{dir_name}_{chr_idx:05d}_{annotate[0]}-{annotate[1]}-{annotate[2]}_.png', 'wb') as f:
                                encoded_img.tofile(f) 
                        vocab.add(annotate[0])
                        vocab.add(annotate[1])
                        vocab.add(annotate[2]+'_') 
                        chr_idx += 1 
                        continue                     
                    elif len(annotate) >= 4:
                        continue
                    else:
                        pass
                    jamo_str = j2hcj(h2j(annotate))
                    if len(jamo_str) == 2:
                        extention = '.png'
                        result, encoded_img = cv2.imencode(extention, pp)
                        if result:
                            if jamo_str[0] in excep_word or jamo_str[1] in excep_word:
                                continue
                            with open(f'/content/data/크랍이미지/{dir_name}_{chr_idx:05d}_{jamo_str[0]}-{jamo_str[1]}-b_.png', 'wb') as f:
                                encoded_img.tofile(f)
                        vocab.add(jamo_str[0])
                        vocab.add(jamo_str[1])
                        vocab.add('b_')
                        chr_idx += 1                        
                    elif len(jamo_str) == 3:
                        extention = '.png'
                        result, encoded_img = cv2.imencode(extention, pp)
                        if result:
                            if jamo_str[0] in excep_word or jamo_str[1] in excep_word or jamo_str[2] in excep_word:
                                continue
                            with open(f'/content/data/크랍이미지/{dir_name}_{chr_idx:05d}_{jamo_str[0]}-{jamo_str[1]}-{jamo_str[2]}_.png', 'wb') as f:
                                encoded_img.tofile(f)
                        vocab.add(jamo_str[0])
                        vocab.add(jamo_str[1])
                        vocab.add(jamo_str[2]+'_') 
                        chr_idx += 1  
                    else:
                        continue         
                    # plt.imsave(f'/content/data/크랍이미지/{dir_name}_{chr_idx:03d}_{jamo_str}.png', pp)

                    # print('saved')

                except:
                    exception.add(annotate)
                    # print('exception')

                # print(jamo_str)
            # plt.show()
            # plt.imshow(pp)
            # plt.show()

In [ ]:
len(glob('/content/data/크랍이미지/*')) # 56 기준 : 5861 너무 적다. 32 기준 : 9861 너무 적다. / extend a, b 허용 56기준

744788

In [ ]:
len(vocab)

192

In [ ]:
list(vocab)

In [ ]:
real_vocab = vocab - set(ll)

In [ ]:
sorted(list(real_vocab))

In [ ]:
with open('/content/token.txt', 'w') as f:
    for i in sorted(list(real_vocab)):
        f.write(i+'\n')

/content


In [ ]:
for i in glob('/content/data/크랍이미지/*'):
    for j in ll:
        if j in i[-10:-4]:
            print(i)
            break

In [ ]:
cnt = 0
for i in real_vocab:
    if '^' in i:
        cnt += 1
cnt

76

In [ ]:
len(real_vocab)

187

In [ ]:
ll = ['풍', '날','젼','광','황','(']

In [8]:
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="rkd954115@")

In [ ]:
while True:
    a=1

In [ ]:
%cd '/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/recognition'

/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/recognition


In [ ]:
import warnings
from importlib import import_module
import wandb

parser = {
    'config_file': './test.yaml'
}

In [ ]:
!pip install tqdm